# Library

In [10]:
from beb_chargers.scripts.script_helpers import build_trips_df, \
    build_charger_location_inputs
from beb_chargers.opt.charger_location import ChargerLocationModel
from beb_chargers.gtfs_beb import GTFSData
from beb_chargers.vis import plot_trips_and_terminals, plot_deadhead
from pathlib import Path
import datetime
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

# Parameters

In [11]:
# Power output of each charger. Note that the model expects a unit of
# kilowatt-hours per minute, so that the amount of energy gained equals
# charging time (in minutes) multiplied by power (in kWh/min). Here, we
# use 450 kW = 450 / 60 kWh/min
chg_pwrs = 450 / 60

# Maximum number of chargers per site. We'll set it to 4 everywhere
n_max = 4

# Cost parameters. See TRC paper for where the values come from.
s_cost = 200000
c_cost = 698447
alpha = 190 * 365 * 12 / 60

# Locate CSV file giving candidate charger sites for this instance
site_fname = Path.cwd().parent / 'beb_chargers' / 'data' / 'test.csv'

# Load candidate charging sites given by Metro and add params as columns
loc_df = pd.read_csv(site_fname)
loc_df['max_chargers'] = n_max
loc_df['kw'] = chg_pwrs * 60
loc_df['fixed_cost'] = s_cost
loc_df['charger_cost'] = c_cost

# Define coordinates of overnight depot at South Base
depot_coords = (40.819197, -73.957060)

# Data

In [12]:
# Directory to GTFS files, as a platform-agnostic path
gtfs_dir = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS'

# Load GTFS data into our custom object
gtfs = GTFSData.from_dir(gtfs_dir)

In [13]:
# Battery capacity in kWh
battery_cap = 300
# Energy consumption per mile for all buses (we'll assume it's the same
# for all buses here)
kwh_per_mi = 3

In [14]:
import csv
import random

In [15]:
import csv
import random

def read_routes_file(filename):
    """
    Reads the routes file and returns a list of route IDs that start with 'M'.
    
    Args:
        filename (str): Path to the CSV file containing route IDs.
    
    Returns:
        list: Filtered list of route IDs where the first letter is 'M'.
    """
    route_ids = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # if row['route_id'].startswith('M'):  # Filter routes that start with 'M'
            #    route_ids.append(row['route_id'])
            route_ids.append(row['route_id'])
    return route_ids

def random_select_routes(route_ids, percentage=.1):
    """
    Randomly selects a subset of routes from the filtered list.
    
    Args:
        route_ids (list): List of filtered route IDs.
        percentage (float): Percentage of routes to select per iteration.
    
    Returns:
        list: Randomly selected route IDs.
    """
    num_to_select = round(len(route_ids) * percentage)
    return random.sample(route_ids, num_to_select) if route_ids else []

def main():
    filename = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS' / 'routes.txt'
    num_iterations = 10  # Define the number of iterations
    
    route_ids = read_routes_file(filename)  # Read and filter routes
    print(f"Total number of routes starting with 'M': {len(route_ids)}")
    
    all_results = []
    
    for i in range(num_iterations):
        selected = random_select_routes(route_ids)  # Select 20% of the filtered routes
        all_results.append(selected)
        print(f"\nIteration {i+1}:")
        print(f"Selected {len(selected)} routes: {selected}")
    
    return all_results

if __name__ == "__main__":
    random.seed(42)  # Ensure reproducibility
    
    # Run the main process
    results = main()
    
    print("\nFinal 2D array of selected route_ids:")
    for i, row in enumerate(results):
        print(f"Row {i+1}: {row}")

Total number of routes starting with 'M': 314

Iteration 1:
Selected 31 routes: ['B92', 'B26', 'M125', 'M4', 'OHAS', 'BX12+', 'B83', 'L90', 'B68', 'B93', 'SIM7', 'B35', 'B32', 'B70', 'MVAS', 'CSAS', 'S89', 'Q97', 'B3', 'Q108', 'BX41+', 'SIM5', 'MQAS', 'SIM31', 'BX90', 'M14A+', 'B13', 'BX20', 'Q15A', 'BX19', 'GHAS']

Iteration 2:
Selected 31 routes: ['Q13', 'B83', 'B70', 'Q56', 'B8', 'Q30', 'Q17', 'Q121', 'M104', 'B41', 'S40', 'J99', 'BX6+', 'Q55', 'B63', 'Q92', 'M31', 'Q32', 'F1', 'BX39', 'B57', 'B42', 'FBAS', 'M22', 'CSAS', 'B82+', 'M14A+', 'SIM33C', 'Q36', 'BX22', 'Q44+']

Iteration 3:
Selected 31 routes: ['Q27', 'SHNRD', 'M106', 'B6', 'B106', 'BX27', 'D99', 'M4', 'BX22', 'S42', 'Q56', 'M116', 'Q95', 'MQAS', 'Q1', 'B46', 'GAAS', 'B35', 'M79+', 'Q110', 'M11', 'B52', 'SIR5', 'Q96', 'S81', 'Q84', 'SIM35', 'BX15', 'M104', 'BX12+', 'M5']

Iteration 4:
Selected 31 routes: ['Q108', 'J90', 'M103', 'S96B3', 'SIM9', 'S96B2', 'Q88', 'Q32', 'MQAS', 'BX12', 'S91', 'S76', 'B7', 'B43', 'B91A', 'BX1

In [16]:
results = main()

Total number of routes starting with 'M': 314

Iteration 1:
Selected 31 routes: ['M4', 'Q44+', 'M15+', 'BX2', 'SIM23', 'L92', 'M42', 'B111', 'X63', 'B14', 'Q93', 'M35', 'B84', 'BX10', 'M104', 'BX4', 'B9', 'BX19', 'M12', 'M15', 'Q121', 'SHNRD', 'Q16', 'BX99', 'S89', 'S66', 'M8', 'B44+', 'B70', 'SIM7', 'M141A']

Iteration 2:
Selected 31 routes: ['B41', 'B11', 'Q5', 'BX9', 'M103', 'BX21', 'SIM25', 'Q92', 'SIM8X', 'Q108', 'B14', 'B92', 'B61', 'BX18', 'L90', 'B37', 'Q44+', 'S96B2', 'BX17', 'SIM10', 'BX8', 'B4', 'M57', 'Q36', 'B39', 'Q30', 'SHNRD', 'M7', 'B83', 'Q27', 'Q107']

Iteration 3:
Selected 31 routes: ['SIM2', 'BX19', 'WFAS', 'BX22', 'BX30', 'SIM4', 'B26', 'BX31', 'Q5', 'SIM3C', 'M7', 'M106', 'BX20', 'B91', 'Q58', 'B38', 'S51', 'OFAS', 'BX42', 'S40', 'Q24', 'M55', 'FBAS', 'OHAS', 'BX39', 'Q88', 'Q3', 'M14A+', 'B57', 'S91', 'J99']

Iteration 4:
Selected 31 routes: ['Q4', 'B31', 'BX4', 'M102', 'BX31', 'S96B1', 'M104', 'B38', 'B91', 'B101', 'SIM15', 'Q17', 'M72', 'SIM22', 'Q109', 'S92',

In [17]:
# All routes included in analysis
beb_routes = [
    'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M9', 'M10',
    'M100', 'M101', 'M102', 'M103', 'M104',
    'M14A-SBS', 'M14D-SBS', 'M15-SBS', 'M60-SBS',
    'Q30', 'Bx18B', 'Bx41-SBS', 'Bx36', 'B44-SBS',
    'B52', 'B74', 'X37', 'B111'
]
beb_routes = [str(r) for r in beb_routes]

beb_routes = results[0]


import csv
import random

def read_routes_file(filename):
    """
    Read routes file and return route_ids list
    Args:
        filename: CSV path
    Returns:
        route_ids list
    """
    route_ids = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            route_ids.append(row['route_id'])
    return route_ids

def filter_m_routes(route_ids):
    """
    Filter routes that start with 'M'
    Args:
        route_ids: list of all route IDs
    Returns:
        list of route IDs starting with 'M'
    """
    return [route for route in route_ids if route.startswith('M')]

def random_select_routes_with_m_first(route_ids, percentage=0.1):
    """
    Select routes with first one being M-route
    Args:
        route_ids: all routes ID
        percentage: percentage to select
    Returns:
        selected routes ID list starting with an M-route
    """
    # Separate M routes and non-M routes
    m_routes = filter_m_routes(route_ids)
    non_m_routes = [r for r in route_ids if not r.startswith('M')]
    
    if not m_routes:
        raise ValueError("No M-routes found in the input data")
    
    # Calculate how many additional routes to select after M-route
    total_to_select = round(len(route_ids) * percentage)
    remaining_to_select = total_to_select - 1  # -1 because we'll select one M-route
    
    # Select one M-route for the first position
    selected_routes = [random.choice(m_routes)]
    
    # Select remaining routes from all available routes (excluding the selected M-route)
    available_routes = [r for r in route_ids if r != selected_routes[0]]
    
    # Randomly select the remaining routes
    for _ in range(remaining_to_select):
        if not available_routes:
            break
        idx = random.randrange(len(available_routes))
        selected_routes.append(available_routes.pop(idx))
    
    return selected_routes

def main():
    filename = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS' / 'routes.txt'
    num_iterations = 10
    
    route_ids = read_routes_file(filename)
    print(f"Total number of routes: {len(route_ids)}")
    print(f"Number of M-routes: {len(filter_m_routes(route_ids))}")
    
    all_results = []
    
    for i in range(num_iterations):
        selected = random_select_routes_with_m_first(route_ids)
        all_results.append(selected)
        print(f"\nIteration {i+1}:")
        print(f"Selected {len(selected)} routes: {selected}")
        print(f"First route starts with 'M': {selected[0].startswith('M')}")
    
    return all_results

if __name__ == "__main__":
    random.seed(42)
    
    # Run main program
    results = main()
    
    print("\nFinal 2D array of selected route_ids:")
    for i, row in enumerate(results):
        print(f"Row {i+1}: {row}")

In [18]:
# We'll run analysis for March 28, 2024
ocl_date = datetime.datetime(2025, 3, 14)
# Call helper function to build up DF
beb_trips = build_trips_df(
    gtfs=gtfs,
    date=ocl_date,
    routes=beb_routes,
    depot_coords=depot_coords,
    add_depot_dh=True,
    add_kwh_per_mi=False,
    add_durations=False,
    routes_60=[]
)
# Add a column giving energy consumption for each trip
beb_trips['kwh_per_mi'] = kwh_per_mi

INFO:gtfs_data:OSM request: 30 origins, 35 destinations (1050 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 1.00 seconds.


In [19]:
beb_trips

,trip_id,route_id,service_id,block_id,shape_id,route,route_type,route_desc,start_time,end_time,trip_idx,start_lat,start_lon,end_lat,end_lon,total_dist,duration_sched,60_dummy,kwh_per_mi
0,OH_A5-Saturday-BM-141800_M15_201,M15,OH_A5-Saturday-BM,36440938,M150023,M15,3,via 1st Av / 2nd Av,2025-03-14 23:38:00,2025-03-15 00:39:00,0,40.803182,-73.932486,40.701549,-74.012199,20.429058,61.0,0,3
1,MV_A5-Saturday-BM-139500_M104_101,M104,MV_A5-Saturday-BM,36441665,M1040167,M104,3,via Broadway / 8th Av,2025-03-14 23:15:00,2025-03-14 23:56:00,0,40.814222,-73.953091,40.756384,-73.989832,11.766449,41.0,0,3
2,OH_A5-Weekday-SDon-006000_M15_201,M15,OH_A5-Weekday-SDon,36441933,M150044,M15,3,via 1st Av / 2nd Av,2025-03-14 01:00:00,2025-03-14 01:43:00,0,40.701481,-74.012461,40.803102,-73.932299,18.574302,43.0,0,3
3,OH_A5-Weekday-SDon-012000_M15_201,M15,OH_A5-Weekday-SDon,36441933,M150023,M15,3,via 1st Av / 2nd Av,2025-03-14 02:00:00,2025-03-14 02:46:00,1,40.803182,-73.932486,40.701549,-74.012199,8.641575,46.0,0,3
4,OH_A5-Weekday-SDon-018000_M15_201,M15,OH_A5-Weekday-SDon,36441933,M150044,M15,3,via 1st Av / 2nd Av,2025-03-14 03:00:00,2025-03-14 03:43:00,2,40.701481,-74.012461,40.803102,-73.932299,8.609989,43.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,QV_A5-Weekday-SDon-104500_X63_757,X63,QV_A5-Weekday-SDon,36882921,X630036,X63,3,via Merrick Bl / Francis Lewis Bl,2025-03-14 17:25:00,2025-03-14 19:17:00,0,40.737263,-73.979386,40.652763,-73.736869,51.960957,112.0,0,3
2261,QV_A5-Weekday-SDon-106000_X63_758,X63,QV_A5-Weekday-SDon,36882922,X630036,X63,3,via Merrick Bl / Francis Lewis Bl,2025-03-14 17:40:00,2025-03-14 19:31:00,0,40.737263,-73.979386,40.652763,-73.736869,51.960957,111.0,0,3
2262,QV_A5-Weekday-SDon-110000_X63_759,X63,QV_A5-Weekday-SDon,36882923,X630036,X63,3,via Merrick Bl / Francis Lewis Bl,2025-03-14 18:20:00,2025-03-14 20:05:00,0,40.737263,-73.979386,40.652763,-73.736869,51.960957,105.0,0,3
2263,QV_A5-Weekday-SDon-114000_X63_760,X63,QV_A5-Weekday-SDon,36882924,X630036,X63,3,via Merrick Bl / Francis Lewis Bl,2025-03-14 19:00:00,2025-03-14 20:28:00,0,40.737263,-73.979386,40.652763,-73.736869,51.960957,88.0,0,3


In [20]:
# Record how many trips and blocks are active on our specified routes on this day
logging.info(
    '{}: There are {} total trips to be served by {} BEB blocks.'.format(
        ocl_date.strftime('%m/%d/%y'), len(beb_trips), beb_trips['block_id'].nunique()
    )
)

INFO:root:03/14/25: There are 2265 total trips to be served by 336 BEB blocks.


In [21]:
# Create a map of the problem instance
inst_map = plot_trips_and_terminals(
    beb_trips, loc_df, gtfs.shapes_df, 'light'
)
# These config params make map downloads higher resolution
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'scale': 3
    }
}
inst_map.show(config=config)

In [22]:
opt_kwargs = build_charger_location_inputs(
    gtfs=gtfs,
    trips_df=beb_trips,
    chargers_df=loc_df,
    depot_coords=depot_coords,
    battery_cap=battery_cap
)

INFO:gtfs_data:OSM request: 21 origins, 38 destinations (798 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 0.95 seconds.
INFO:gtfs_data:OSM request: 38 origins, 24 destinations (912 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 0.95 seconds.
INFO:gtfs_data:OSM request: 39 origins, 45 destinations (1755 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 1.07 seconds.


In [23]:
# this is the dictionary
type (opt_kwargs)

dict

In [24]:
# Create instance of charger location model based on those inputs
clm = ChargerLocationModel(**opt_kwargs)
# Solve the model with zero optimality gap
clm.solve(alpha=alpha, opt_gap=0, bu_kwh=battery_cap)

INFO:charger_location:Number of blocks that require charging: 166
INFO:charger_location:Number of trips in charging blocks: 1389
INFO:charger_location:Number of infeasible blocks: 132
INFO:charger_location:Infeasible block IDs: ['36459509', '36459510', '36459511', '36459512', '36459517', '36459520', '36459521', '36459522', '36459523', '36459537', '36459538', '36459692', '36459693', '36459694', '36459695', '36459696', '36459697', '36459698', '36459699', '36459700', '36459701', '36459702', '36459703', '36459704', '36459705', '36459706', '36459707', '36459708', '36459709', '36459710', '36459711', '36459712', '36459713', '36459714', '36459715', '36459716', '36459717', '36459718', '36459719', '36459720', '36459722', '36459723', '36459724', '36461209', '36461210', '36461211', '36461213', '36461214', '36461215', '36461216', '36461217', '36461218', '36461219', '36461220', '36461221', '36461222', '36461224', '36461258', '36461259', '36461261', '36461262', '36461263', '36461264', '36461265', '36

In [25]:
clm.log_results()

INFO:charger_location:Total number of backup buses used: 169
INFO:charger_location:Optimal objective function value: 12581117.43
INFO:charger_location:Optimal stations: ['3rd Avenue138th Street Station', 'Brooklyn College / Flatbush Avenue Terminal', 'Dyckman Street Station', 'Flushing  Main Street', 'Gateway Center (Near Spring Creek)', 'St. George Ferry Terminal', 'Staten Island Mall', 'Station 40']
INFO:charger_location:Number of chargers: {'149th StreetGrand Concourse': 0, '161st StreetYankee Stadium Station': 0, '167th Street Station': 0, '174th175th Streets Station': 0, '182nd183rd Streets Station': 0, '207th Street Station': 0, '215th Street Station': 0, '3rd Avenue138th Street Station': 1, 'Atlantic Terminal': 0, 'Brooklyn College / Flatbush Avenue Terminal': 2, 'Bus Stops around LaGuardia Airport': 0, 'Co-op City Bay Plaza': 0, 'Coney Island  Stillwell Avenue Terminal': 0, 'Dyckman Street Station': 1, 'Eltingville Transit Center': 0, 'Far Rockaway  Mott Avenue': 0, 'Flushing  

In [26]:
clm.to_df()

,block_id,trip_id,trip_idx,start_time,end_time,soc,chg_site,chg_time,dh1,dh2,dh3
0,36441937,OH_A5-Weekday-SDon-023000_M15_205,1,65848550.0,65848600.0,293.272896,NaN,NaN,NaN,NaN,NaN
1,36441937,OH_A5-Weekday-SDon-029000_M15_205,2,65848610.0,65848657.0,260.578257,NaN,NaN,NaN,NaN,NaN
2,36441937,OH_A5-Weekday-SDon-035000_M15_205,3,65848670.0,65848721.0,234.714226,NaN,NaN,NaN,NaN,NaN
3,36441937,OH_A5-Weekday-SDon-040700_M15_205,4,65848727.0,65848791.0,212.262560,3rd Avenue138th Street Station,1.791611,4.425333,4.641167,0.011355
4,36441937,OH_A5-Weekday-SDon-049100_M15_205,5,65848811.0,65848896.0,197.110666,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
517,36879652,100,23,65849466.0,65849466.0,45.152741,NaN,NaN,NaN,NaN,NaN
518,36879662,0,0,65848747.0,65848747.0,300.000000,NaN,NaN,NaN,NaN,NaN
519,36879662,100,24,65849635.0,65849635.0,40.804363,NaN,NaN,NaN,NaN,NaN
520,36879674,0,0,65848742.0,65848742.0,300.000000,NaN,NaN,NaN,NaN,NaN


In [27]:
# clm.plot_chargers()

In [28]:
fig = plot_deadhead(
    result_df=clm.to_df(), loc_df=loc_df, coords_df=beb_trips
)
fig.show()

INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
